# Важно - не забывайте закрывать сессию. Если у вас идёт запрос и из питона, и из терадаты к одной таблице это приведёт к зависанию и проблемам

## Турбодбс может не работать, если у вас не в том порядке установлены библиотеки - чтобы исправить эту ошибку скопируйте environment с гитхаба (главная страница внизу)

In [1]:
import turbodbc
import numpy as np
import pandas as pd

In [2]:
sql = """
   sel  SEGMENT, MSISDN, report_date, SUBS_ID, 
        sd_day_voice_cnt,
days_with_data_trafic,
cpe_popularity,av_dev_use,
avg_ses_mou,
bal_avg_amt,
bal_min_amt,
sd_day_mbou,
cpe_m_age,
bal_max_amt,
days_home_cnt,
avg_day_mbou,
avg_day_voice_cnt,
sms_tot_cnt,
out_data_ratio,
sum_ses_mbou,
cpe_p_age,
avg_day_data_cnt,
cl_opp_gender_share,
dev_park_size,
cl_core_opp_gender_share,
lifetime,
ret_dev_ind,
max_dev_use,
cl_avg_lifetime
 from UAT_DM.TE_MS_20180202_MRT
 where 
  cpe_type in ('C2','C3')
 and (sum_ses_mbou>0 or avg_day_mou>0)
 and lifetime>30"""

In [3]:
from turbodbc import connect
connection = connect(dsn='Teradata')

In [4]:
cursor = connection.cursor()

In [5]:
cursor.execute(sql)

In [6]:
%%time
data = pd.DataFrame(cursor.fetchallnumpy())

Wall time: 1.99 s


In [7]:
print(data.shape)
data.head()

(21088, 29)


,SEGMENT,MSISDN,report_date,SUBS_ID,sd_day_voice_cnt,days_with_data_trafic,cpe_popularity,av_dev_use,avg_ses_mou,bal_avg_amt,...,sum_ses_mbou,cpe_p_age,avg_day_data_cnt,cl_opp_gender_share,dev_park_size,cl_core_opp_gender_share,lifetime,ret_dev_ind,max_dev_use,cl_avg_lifetime
0,3,79527509799,2017-03-01,100053038055,26.878244,30,4.805847,10.292683,31.757333,5.295455,...,1422,2,22.266667,0.36,6,0.93,33,1.00,363,101.357143
1,4,79511363663,2017-11-01,50118600,67.108072,30,4.551194,231.000000,58.498762,39.892308,...,7894,86,118.566667,0.00,4,0.00,114,0.94,989,1954.107143
2,5,79537089356,2017-12-01,26224371,7.838367,0,5.269861,1683.000000,657.012658,86.019167,...,0,1683,0.000000,0.00,1,0.00,2918,0.34,1683,1476.928571
3,5,79517010180,2017-11-01,200050099804,8.934328,1,5.424032,172.400000,186.805310,171.542308,...,9,6,1.000000,0.17,9,0.33,1073,0.99,921,1415.333333
4,3,79529198075,2017-02-01,100021553622,10.450359,2,4.209676,1775.000000,82.297170,93.835714,...,1,1775,1.500000,0.43,1,0.85,2631,0.06,1775,2188.571429


## Ниже - пример как делать инсёрт. Обратите внимание, что он работает не со всеми типами данных (например, date напрямую не получится закинуть, нужно бить на интеджеры
Здесь идёт инсёрт двух колонок (индекс и мсисдн), для большего числа логика ровно такая же
## После инсёрта не забываем коммит (чтобы изменения сохранились) и закрываем сессию

In [7]:
%%time
cursor.executemanycolumns("INSERT INTO UAT_DM.AR_TURBODBC_TEST VALUES (?, ?)",
                          [data.index.values, data['MSISDN'].values])
connection.commit()
connection.close()
print("done")

done
Wall time: 10min 26s
